C# and .NET define ***two standard protocols*** for determining the `order` of `one object` ***relative*** to `another`  
- The `IComparable` interfaces (`IComparable` and `IComparable<T>`)
- The `>` and `<` operators

The `IComparable` interfaces are used by ***general-purpose*** `sorting` algorithms.

In [ ]:
//the static Array.Sort method works because System.String
//implements the IComparable interfaces

string[] colors = { "Green", "Red", "Blue" };
Array.Sort (colors);
foreach (string c in colors) Console.Write (c + " "); // Blue Green Red

### IComparable

In [ ]:
public interface IComparable 
{ 
    int CompareTo (object other); 
}

public interface IComparable<in T> 
{ 
    int CompareTo (T other); 
}

the ***generic type-safe interface*** is `faster` than the ***nongeneric interface***.

In both cases, the Compare To method works as follows
- If `a` comes **after** `b`, ***a.CompareTo(b)*** returns a `positive` number.
- If `a` is the **same as** `b`, ***a.CompareTo(b)*** returns `0`.
- If `a` comes **before** `b`, ***a.CompareTo(b)*** returns a `negative` number.

In [1]:
Console.WriteLine ("Beck".CompareTo ("Anne")); // 1
Console.WriteLine ("Beck".CompareTo ("Beck")); // 0
Console.WriteLine ("Beck".CompareTo ("Chris")); // -1

1
0
-1


#### IComparable versus Equals

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p><strong>رفتار مورد انتظار:</strong></p>
<ul><li>هنگامی که یک نوع هم متد <code>Equals</code> را بازنویسی کرده و هم رابط <code>IComparable</code> را پیاده‌سازی می‌کند، انتظار داریم که اگر <code>Equals</code> مقدار true را برگرداند، <code>CompareTo</code> باید مقدار 0 را برگرداند. به عبارت دیگر، اگر دو شیء برابر هستند، مقایسه آن‌ها نیز باید نشان‌دهنده برابر بودن آن‌ها باشد.</li><li>اما هنگامی که <code>Equals</code> مقدار false را برمی‌گرداند، <code>CompareTo</code> می‌تواند هر مقداری را برگرداند، به شرطی که در داخل نوع سازگار باشد.</li></ul>

<ul><li><code>Equals</code> می‌تواند در تشخیص برابری دقیق‌تر و حساس‌تر باشد، اما برعکس آن ممکن نیست. اگر <code>CompareTo</code> نسبت به <code>Equals</code> حساس‌تر باشد، الگوریتم‌های مرتب‌سازی خراب می‌شوند.</li><li>بنابراین، <code>CompareTo</code> می‌تواند همه اشیاء را برابر بداند، در حالی که <code>Equals</code> می‌تواند بگوید "برخی برابرتر از دیگران هستند."</li></ul>
</div>

### `<` and `>`

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>پیاده‌سازی عملگرهای <code>&lt;</code> و <code>&gt;</code> در انواعی که دارای مفهوم ذاتی قوی از "بزرگتر بودن" و "کوچکتر بودن" هستند، منطقی است. این عملگرها باید با <code>IComparable</code> سازگار باشند تا عملکرد صحیح مقایسه‌ها تضمین شود. از سوی دیگر، پیاده‌سازی این عملگرها در انواعی که مقایسه آن‌ها ممکن است بسته به فرهنگ و زبان متفاوت باشد، منطقی نیست و می‌تواند منجر به رفتار غیرمنتظره شود.</p>

<p><strong>شرایطی برای بازنویسی <code>&lt;</code> و <code>&gt;</code>:</strong></p>
<ul><li>معمولاً عملگرهای <code>&lt;</code> و <code>&gt;</code> فقط زمانی بازنویسی می‌شوند که:<ul><li>نوع دارای مفهوم ذاتی قوی از "بزرگتر بودن" و "کوچکتر بودن" باشد (در مقایسه با مفاهیم وسیع‌تر <code>IComparable</code> از "قبل آمدن" و "بعد آمدن").</li><li>تنها یک روش یا زمینه برای انجام مقایسه وجود داشته باشد.</li><li>نتیجه مقایسه در تمام فرهنگ‌ها یکسان باشد.</li></ul></li></ul>

<p>نوع <code>System.String</code> این شرایط را برآورده نمی‌کند. نتایج مقایسه رشته‌ها می‌توانند بسته به زبان متفاوت باشند.</p>
</div>

In [ ]:
bool error = "Beck" > "Anne"; // خطای زمان کامپایل

In [ ]:
public struct Note : IComparable<Note>, IEquatable<Note>, IComparable
{
    private int _semitonesFromA;
    public int SemitonesFromA { get { return _semitonesFromA; } }

    public Note(int semitonesFromA)
    {
        _semitonesFromA = semitonesFromA;
    }

    public int CompareTo(Note other) // Generic IComparable<T>
    {
        if (Equals(other)) return 0; // Fail-safe check
        return _semitonesFromA.CompareTo(other._semitonesFromA);
    }

    int IComparable.CompareTo(object other) // Nongeneric IComparable
    {
        if (!(other is Note))
            throw new InvalidOperationException("CompareTo: Not a note");
        return CompareTo((Note)other);
    }

    public static bool operator <(Note n1, Note n2)
    => n1.CompareTo(n2) < 0;

    public static bool operator >(Note n1, Note n2)
    => n1.CompareTo(n2) > 0;

    public bool Equals(Note other) // for IEquatable<Note>
    => _semitonesFromA == other._semitonesFromA;

    public override bool Equals(object other)
    {
        if (!(other is Note)) return false;
        return Equals((Note)other);
    }
    public override int GetHashCode() => _semitonesFromA.GetHashCode();

    public static bool operator ==(Note n1, Note n2) => n1.Equals(n2);

    public static bool operator !=(Note n1, Note n2) => !(n1 == n2);
}